# Libraries importation

In [1]:
# Import all libraries needed 

import matplotlib.pyplot as plt
import pandas as pd
import sys 
import scipy
import numpy as np

from scipy.signal import *
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics

In [2]:
sys.path.insert(0, '../packaging_PIR')
from neural_data_treatment_pkg.PrintFunctions import *
from neural_data_treatment_pkg.AdaBandFlt import *

In [3]:
# Enable inline plotting
%matplotlib inline
# Enable outline plotting
%matplotlib tk

# Data Loading (execute only the one corresponding to your device)

In [4]:
# file path of csv file

Location = r'/Users/sylva/Documents/SUPAERO/2A/PIR/Data/data_no_burst/539W6cbaseRaw.txt'

In [5]:
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

In [6]:
# complete this one with your own path

#Location = 

In [5]:
# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

C:\Users\sylva\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
df.columns

Index(['El 41       ', 'El 42       ', 'El 44       ', 'El 54       ',
       'El 55       ', 'El 56       ', 'El 58       ', 'El 86       ',
       'El 15       ', 'El 51       '],
      dtype='object')

# Defining the sampling frequency and the alignement method for the rest of the notebook

In [90]:
fs = 50000
align_method = 'indice_1er_depass'
y_lim_min = -0.5
y_lim_max = 0.5
time_before = 0.0015
time_after = 0.0015

n_electrode = 4
size_of_data = 1000000

# Cuting and filtering the signal

In [12]:
lowcut = 100.0
highcut = 5000.0

filtered_signal = []

for electrode in range(n_electrode):
    before_filter = df.iloc[:size_of_data,electrode]
    
    y = butter_bandpass_filter(before_filter, lowcut, highcut, fs, order=6)
    
    filtered_signal.append(y)
    
filtereddf_not_transposed = pd.DataFrame(filtered_signal)

    
filtereddf = filtereddf_not_transposed.transpose()
filtereddf.index = df.index[:size_of_data]
filtereddf.columns = df.columns[:n_electrode]

filtereddf

C:\Users\sylva\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


,El 41,El 42,El 44,El 54
%t,,,,
0.00,-0.670835,1.045904,0.452347,1.051048
0.02,-0.543104,1.128120,-0.302855,0.686790
0.04,-0.411112,1.238128,-0.994273,0.316140
0.06,-0.284953,1.412100,-1.569000,-0.031117
0.08,-0.195775,1.696732,-1.993970,-0.272956
0.10,-0.189916,2.141402,-2.262628,-0.289464
0.12,-0.312198,2.781215,-2.396996,0.035509
0.14,-0.584963,3.616421,-2.443208,0.761599
0.16,-0.991420,4.596725,-2.459831,1.850652


# Detecting and Aligning the Spikes

### Setting Up Thresholds Adapted to Noise Levels for Each Electrode

In [13]:
noise_levels_all_electrodes = []

for electrode in range(n_electrode):
    noise_levels = init_noise_levels(filtereddf.iloc[:size_of_data, electrode], fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 20,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)
    noise_levels_all_electrodes.append(noise_levels)


In [14]:
# Plotting the noise levels for all electrodes
plt.figure()
for electrode in range(n_electrode):
    plt.plot(noise_levels_all_electrodes[electrode])
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5,1,'Noise Levels')

### Detecting Spikes for Each Electrode

In [15]:
list_spike_info = []

for electrode in range(n_electrode):
    spike_info = find_spikes(filtereddf.iloc[:size_of_data, electrode], noise_levels_all_electrodes[electrode], fs, 
                          window_size = 0.001, 
                          noise_window_size = 0.01,
                          threshold_factor = 3.0,
                          maxseparation = 0.002)
    spike_info['Electrode num'] = electrode
    #Normalizing the amplitudes in a new column
    amplitude_norm = np.linalg.norm(spike_info['Delta_amplitudes'])
    spike_info['Delta_amp_norm'] = spike_info['Delta_amplitudes']/amplitude_norm
    list_spike_info.append(spike_info)

In [16]:
list_spike_info

[     indice_min  indice_1er_depass indice_max_gauche indice_max_droite  \
 0         10959              10959             10934             10968   
 1         12439              12439               nan             12482   
 2         13728              13727             13708             13798   
 3         14075              14073             14066               nan   
 4         14194              14193             14152               nan   
 5         14858              14857             14765               nan   
 6         15226              15225             15187               nan   
 7         15377              15377             15341             15448   
 8         16046              16045               nan             16098   
 9         19094              19094             19042             19129   
 10        23967              23967             23921               nan   
 11        28176              28173             28132               nan   
 12        32132         

### Recording the Spikes For Each Electrode

In [54]:
list_spike_data = []

for electrode in range(n_electrode):
    spike_data = record_spikes(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                           align_method,
                           t_before = time_before, 
                           t_after = time_after)
    list_spike_data.append(spike_data)

list_spike_data

[          0         1         2         3         4         5         6    \
 0    3.278505 -3.968054 -1.916645 -0.327927  1.044481 -0.624443  2.250833   
 1    3.346240 -3.261710 -1.942784 -0.339991  0.439007 -0.715764  3.294266   
 2    3.041488 -2.259862 -2.125253 -0.631809  0.190159 -0.482692  3.984122   
 3    2.459027 -1.169752 -2.366511 -1.151664  0.296905  0.014270  4.199057   
 4    1.749595 -0.187121 -2.560976 -1.786816  0.716802  0.641877  3.894464   
 5    1.081283  0.541478 -2.627228 -2.382535  1.382538  1.233720  3.107636   
 6    0.595867  0.939414 -2.530456 -2.774343  2.217574  1.638810  1.952246   
 7    0.373454  1.001891 -2.288665 -2.828113  3.145552  1.764127  0.602783   
 8    0.416081  0.782524 -1.961809 -2.478480  4.091821  1.598432 -0.730814   
 9    0.654336  0.371320 -1.628647 -1.753933  4.979498  1.210833 -1.837220   
 10   0.973361 -0.128756 -1.359764 -0.778717  5.725215  0.725489 -2.542760   
 11   1.248628 -0.621235 -1.195829  0.252418  6.239964  0.281083

In [67]:
list_spike_data_normalized = []

for electrode in range(n_electrode):
    spike_data_normalized = record_spikes(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                           align_method,
                           t_before = time_before, 
                           t_after = time_after)
    for i in range(len(spike_data_normalized.columns)):
        amp_norm = np.linalg.norm(spike_data_normalized[i])
        spike_data_normalized[i] = spike_data_normalized[i]/amp_norm
    list_spike_data_normalized.append(spike_data_normalized)


In [68]:
list_spike_data_normalized

[          0         1         2         3         4         5         6    \
 0    0.093520 -0.113905 -0.049951 -0.008153  0.029684 -0.019804  0.063510   
 1    0.095452 -0.093629 -0.050632 -0.008453  0.012477 -0.022700  0.092951   
 2    0.086759 -0.064870 -0.055388 -0.015708  0.005404 -0.015308  0.112416   
 3    0.070144 -0.033578 -0.061676 -0.028633  0.008438  0.000453  0.118481   
 4    0.049908 -0.005371 -0.066744 -0.044424  0.020372  0.020357  0.109886   
 5    0.030844  0.015543 -0.068470 -0.059235  0.039292  0.039127  0.087685   
 6    0.016997  0.026966 -0.065948 -0.068976  0.063024  0.051974  0.055085   
 7    0.010653  0.028760 -0.059647 -0.070313  0.089397  0.055948  0.017008   
 8    0.011869  0.022463 -0.051128 -0.061620  0.116290  0.050693 -0.020621   
 9    0.018665  0.010659 -0.042445 -0.043607  0.141518  0.038401 -0.051839   
 10   0.027765 -0.003696 -0.035438 -0.019361  0.162711  0.023008 -0.071747   
 11   0.035617 -0.017833 -0.031165  0.006276  0.177340  0.008914

In [69]:
list_spike_data_oneline = []

for electrode in range(n_electrode):
    spike_data_oneline = record_spikes_oneline(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                                           align_method,
                                           t_before = time_before,
                                           t_after = time_after)
    list_spike_data_oneline.append(spike_data_oneline)

### Plotting the Spikes for Each Electrode

In [70]:
#Plotting the spikes on top of original signal

for electrode in range(n_electrode):
    plt.figure()
    plt.plot(filtereddf.index[:size_of_data], filtereddf.iloc[:size_of_data,electrode], color = 'blue')
    plt.plot(filtereddf.index[:size_of_data], list_spike_data_oneline[electrode], color = 'red')
    plt.title('Filtered Signal with Detected Spikes with RMS for electrode:'+str(filtereddf.columns[electrode]))
    plt.xlabel('Time Windows')
    plt.ylabel('Amplitude [µV]')
    plt.legend()
    plt.grid(True)

In [71]:
from random import randint

def print_spikes_adapted(spike_data,
                 t_before_alignement = 0,
                 first_spike = 1,
                 last_spike = -1,
                 fs = 25000,
                 randomize = False,
                 nb_spike = 20,
                 y_lim_min = -0.5,
                 y_lim_max = 0.5):
    
    if randomize == True:        
        kept = []
        m = len(spike_data.values[0])
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = spike_data.iloc[:,kept].values
        
    else:
        x = spike_data.iloc[:,first_spike:last_spike]
        
    figure = plt.figure()
    t_b = int(np.round(fs*(t_before_alignement)))
    axes = figure.add_subplot(1, 1, 1)
    axes.plot((spike_data.index-t_b)*1000/fs, x)
    axes.set_xlabel('Time in ms')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()

In [72]:
# Plotting aligned spikes for each electrode
for electrode in range(n_electrode):
    print_spikes_adapted(list_spike_data_normalized[electrode],
             t_before_alignement = time_before,
             first_spike = 1,
             last_spike = 50,
             fs = fs)
    plt.title("Spikes aligned on "+str(align_method)+" for electrode "+str(filtereddf.columns[electrode]))

# Bilan PCA + OPTICS

### Merging spike_data and spike_info from all electrodes

In [73]:
spike_data_all_electrodes = pd.concat([list_spike_data[electrode] for electrode in range(n_electrode)], axis=1)
spike_data_all_electrodes.columns = [i+1 for i in range(len(spike_data_all_electrodes.columns))]
spike_data_all_electrodes

,1,2,3,4,5,6,7,8,9,10,...,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168
0,3.278505,-3.968054,-1.916645,-0.327927,1.044481,-0.624443,2.250833,-1.205782,2.665748,1.361656,...,0.640795,1.849628,2.362188,-2.476499,0.529813,2.263155,3.518998,0.596744,-0.562657,0.217757
1,3.346240,-3.261710,-1.942784,-0.339991,0.439007,-0.715764,3.294266,-1.799415,3.295083,1.429836,...,0.606507,1.197423,3.594316,-2.583522,0.780232,3.604009,2.322424,0.064712,-1.212755,-0.706298
2,3.041488,-2.259862,-2.125253,-0.631809,0.190159,-0.482692,3.984122,-2.060916,3.394323,1.365541,...,0.537827,0.388183,4.621814,-2.396218,0.822508,4.791624,0.854920,-0.318045,-1.591489,-1.381583
3,2.459027,-1.169752,-2.366511,-1.151664,0.296905,0.014270,4.199057,-1.977085,2.890396,1.070274,...,0.540149,-0.421234,5.258480,-1.876953,0.628244,5.672519,-0.618072,-0.487370,-1.680709,-1.665485
4,1.749595,-0.187121,-2.560976,-1.786816,0.716802,0.641877,3.894464,-1.612402,1.862907,0.506905,...,0.660648,-1.070176,5.430139,-1.061650,0.206535,6.136860,-1.849901,-0.421808,-1.526733,-1.523865
5,1.081283,0.541478,-2.627228,-2.382535,1.382538,1.233720,3.107636,-1.098259,0.525027,-0.285960,...,0.872958,-1.421624,5.185386,-0.046614,-0.401626,6.141017,-2.655469,-0.151584,-1.227380,-1.039755
6,0.595867,0.939414,-2.530456,-2.774343,2.217574,1.638810,1.952246,-0.600526,-0.830641,-1.200703,...,1.091436,-1.387807,4.670695,1.042313,-1.133949,5.715080,-2.939781,0.247061,-0.905350,-0.388384
7,0.373454,1.001891,-2.288665,-2.828113,3.145552,1.764127,0.602783,-0.274752,-1.905690,-2.087349,...,1.208713,-0.949642,4.079130,2.086430,-1.919250,4.954283,-2.708248,0.671830,-0.674577,0.213631
8,0.416081,0.782524,-1.961809,-2.478480,4.091821,1.598432,-0.730814,-0.223602,-2.469503,-2.792869,...,1.141155,-0.165149,3.589759,3.003822,-2.689223,3.997548,-2.057643,1.023603,-0.609258,0.569373
9,0.654336,0.371320,-1.628647,-1.753933,4.979498,1.210833,-1.837220,-0.470931,-2.414426,-3.201311,...,0.863919,0.837293,3.317170,3.763075,-3.388003,2.999064,-1.149814,1.236796,-0.725107,0.554736


In [74]:
list_spike_data_normalized

spike_data_norm_all_electrodes = pd.concat([list_spike_data_normalized[electrode] for electrode in range(n_electrode)], axis=1)
spike_data_norm_all_electrodes.columns = [i+1 for i in range(len(spike_data_norm_all_electrodes.columns))]
spike_data_norm_all_electrodes

,1,2,3,4,5,6,7,8,9,10,...,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168
0,0.093520,-0.113905,-0.049951,-0.008153,0.029684,-0.019804,0.063510,-0.034446,0.073109,0.048216,...,0.017041,0.062494,0.069743,-0.062486,0.016052,0.063296,0.115137,0.018808,-0.021984,0.006010
1,0.095452,-0.093629,-0.050632,-0.008453,0.012477,-0.022700,0.092951,-0.051404,0.090369,0.050630,...,0.016129,0.040458,0.106122,-0.065187,0.023639,0.100797,0.075987,0.002040,-0.047385,-0.019493
2,0.086759,-0.064870,-0.055388,-0.015708,0.005404,-0.015308,0.112416,-0.058874,0.093091,0.048353,...,0.014303,0.013116,0.136459,-0.060461,0.024920,0.134012,0.027972,-0.010024,-0.062183,-0.038130
3,0.070144,-0.033578,-0.061676,-0.028633,0.008438,0.000453,0.118481,-0.056479,0.079271,0.037898,...,0.014365,-0.014232,0.155256,-0.047359,0.019034,0.158649,-0.020222,-0.015361,-0.065670,-0.045965
4,0.049908,-0.005371,-0.066744,-0.044424,0.020372,0.020357,0.109886,-0.046062,0.051091,0.017949,...,0.017569,-0.036158,0.160324,-0.026787,0.006257,0.171636,-0.060526,-0.013294,-0.059653,-0.042056
5,0.030844,0.015543,-0.068470,-0.059235,0.039292,0.039127,0.087685,-0.031374,0.014399,-0.010126,...,0.023215,-0.048033,0.153098,-0.001176,-0.012168,0.171752,-0.086883,-0.004778,-0.047957,-0.028696
6,0.016997,0.026966,-0.065948,-0.068976,0.063024,0.051974,0.055085,-0.017155,-0.022781,-0.042516,...,0.029025,-0.046890,0.137902,0.026299,-0.034356,0.159840,-0.096186,0.007787,-0.035374,-0.010719
7,0.010653,0.028760,-0.059647,-0.070313,0.089397,0.055948,0.017008,-0.007849,-0.052264,-0.073912,...,0.032144,-0.032086,0.120436,0.052644,-0.058148,0.138562,-0.088610,0.021175,-0.026357,0.005896
8,0.011869,0.022463,-0.051128,-0.061620,0.116290,0.050693,-0.020621,-0.006388,-0.067727,-0.098894,...,0.030347,-0.005580,0.105987,0.075792,-0.081477,0.111804,-0.067323,0.032262,-0.023805,0.015714
9,0.018665,0.010659,-0.042445,-0.043607,0.141518,0.038401,-0.051839,-0.013453,-0.066217,-0.113357,...,0.022975,0.028290,0.097939,0.094949,-0.102648,0.083878,-0.037620,0.038981,-0.028332,0.015310


In [75]:
spike_info_all_electrodes = list_spike_info[0].append([list_spike_info[electrode+1] for electrode in range(n_electrode-1)])
spike_info_all_electrodes.index = [i+1 for i in range(len(spike_info_all_electrodes))]
spike_info_all_electrodes

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes,Electrode num,Delta_amp_norm
1,10959,10959,10934,10968,11.510890,0,0.054478
2,12439,12439,nan,12482,9.597422,0,0.045422
3,13728,13727,13708,13798,14.094262,0,0.066704
4,14075,14073,14066,nan,14.302299,0,0.067689
5,14194,14193,14152,nan,11.447940,0,0.054180
6,14858,14857,14765,nan,12.299364,0,0.058209
7,15226,15225,15187,nan,10.857096,0,0.051384
8,15377,15377,15341,15448,12.943816,0,0.061259
9,16046,16045,nan,16098,11.716995,0,0.055453
10,19094,19094,19042,19129,12.461944,0,0.058979


## PCA and OPTICS on spikes

In [78]:
def PCA_and_AGGLOCLUST_spikes(spike_data, spike_info, nb_PCA_components=3,
                              n_clusters=5, metric='euclidean', linkage='ward'):
    
    ## on rééquilibre les valeurs dans les différentes dimensions
    #pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
    #pca_data = StandardScaler().fit_transform(pca_data) # normalizing the features
    
    ## PCA
    pca_data = np.array(spike_data.values).transpose()
    pca = PCA(n_components=nb_PCA_components)
    pca.fit(pca_data)
    PCA_X = pca.transform(pca_data)
    
    ## AGGLOMERATIVE CLUSTERING
    ## Different linkages: 'ward', 'average', 'complete', 'single'
    
    aggloclustering = AgglomerativeClustering(n_clusters=n_clusters, affinity = metric,
                                    linkage=linkage)
    aggloclustering.fit(PCA_X)
    
    labels = aggloclustering.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    
    ## Ajout du label des clusters dans spike info
    spike_info['cluster_label'] = aggloclustering.labels_
    
    return PCA_X, aggloclustering, spike_info

In [83]:
# Function: PCA_and_OPTICS_spikes(spike_data, spike_info, nb_PCA_components=3,
#                                        min_samples=5, max_eps=10, xi=0.05, min_cluster_size=5)

PCA_X, aggloclustering, updated_spike_info_all_electrodes = PCA_and_AGGLOCLUST_spikes(spike_data_norm_all_electrodes,
                                                                        spike_info_all_electrodes, 
                                                                        nb_PCA_components=3,
                                                                        n_clusters=5, 
                                                                        metric="euclidean", 
                                                                        linkage="ward")


In [84]:
labels = aggloclustering.labels_

## Plotting the PCA

In [85]:
## Fonction qui plot la PCA

def PCA_plot(PCA_X):
    
    fig = plt.figure(4,figsize=(4,3))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    plt.cla()
  
    ax.scatter(PCA_X[:, 0], PCA_X[:, 1], PCA_X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])

    plt.show()

In [86]:
PCA_plot(PCA_X)
print_clusters_3d(labels, PCA_X)

# Print the spikes clusterized

In [91]:
print_spikes_clusterized(spike_data_norm_all_electrodes,
                             labels,
                             t_before_alignement = 0.001,
                             nb_spike = 20,
                             y_lim_min = y_lim_min,
                             y_lim_max = y_lim_max,
                             fs = fs)

In [111]:
i = 0
for key in filtereddf.columns:
    spike_data_clusterized_oneline = record_spikes_clusterized_oneline(filtereddf[key], 
                                                                      fs, 
                                                                      spike_info_all_electrodes.loc[spike_info_all_electrodes['Electrode num'] == i],
                                                                      align_method,
                                                                      labels[spike_info_all_electrodes['Electrode num'] == i],
                                                                      t_before = 0.001,
                                                                      t_after = 0.002)
    print_spikes_clusterized_oneline(filtereddf[key], spike_data_clusterized_oneline,
                                     y_lim_min = y_lim_min,
                                     y_lim_max = y_lim_max,)
    i+=1

The labels located in labels will be used
The labels located in labels will be used
The labels located in labels will be used
The labels located in labels will be used


# Show wich cluster is in wich electrod

In [120]:
spike_info_all_electrodes

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes,Electrode num,Delta_amp_norm,cluster_label
1,10959,10959,10934,10968,11.510890,0,0.054478,4
2,12439,12439,nan,12482,9.597422,0,0.045422,0
3,13728,13727,13708,13798,14.094262,0,0.066704,0
4,14075,14073,14066,nan,14.302299,0,0.067689,0
5,14194,14193,14152,nan,11.447940,0,0.054180,1
6,14858,14857,14765,nan,12.299364,0,0.058209,1
7,15226,15225,15187,nan,10.857096,0,0.051384,3
8,15377,15377,15341,15448,12.943816,0,0.061259,3
9,16046,16045,nan,16098,11.716995,0,0.055453,0
10,19094,19094,19042,19129,12.461944,0,0.058979,4


In [138]:
arrays = []
lab = []
for i in range(n_electrode):
    print(i)
    arrays.append(spike_info_all_electrodes['cluster_label'].loc[spike_info_all_electrodes['Electrode num'] == i])
    lab.append('Electrode ' + str(i))

bins = [x + 0.5 for x in range(0, 6)]
plt.figure()
plt.hist(arrays, label = lab, bins = [x - 0.5 for x in range(0, max(spike_info_all_electrodes['cluster_label']) + 2)],
            histtype = 'bar') # bar est le defaut
plt.ylabel('Spikes number')
plt.xlabel('Clusters')
plt.title('Spikes number by cluster and electrode')
plt.grid(True)
plt.legend()

0
1
2
3
